In [1]:
!pip install gdown -q

In [ ]:
#https://drive.google.com/file/d/1lhAaeQCmk2y440PmagA0KmIVBIysVMwu/view?usp=drive_link

In [2]:
!gdown --id 1lhAaeQCmk2y440PmagA0KmIVBIysVMwu

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu
From (redirected): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&confirm=t&uuid=1eb40f79-8248-4c9c-83a6-9b50136b9e37
To: /content/tennis_court_det_dataset.zip
100% 7.26G/7.26G [01:50<00:00, 65.7MB/s]


In [3]:
!unzip tennis_court_det_dataset.zip -d tennis_court_det_dataset
# !unzip /content/drive/MyDrive/tennis_court_det_dataset.zip -d tennis_court_det_dataset

Streaming output truncated to the last 5000 lines.
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_2200.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_2250.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_250.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_300.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_350.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_400.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_450.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_50.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_500.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_550.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_600.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_650.png  
  inflating: tennis_court_det_dataset/data/images/JNKp7sCdQlY_700.png  
  inflating:

# Training code

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create Dataset

In [5]:
import json
import cv2
import numpy as np

class KeypointDataset(Dataset):
  def __init__(self, img_dir, data_file):
    self.img_dir = img_dir
    self.data = json.load(open(data_file, 'r'))

    self.transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    item = self.data[idx]
    img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
    h,w = img.shape[:2]

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = self.transforms(img)
    kps = np.array(item['kps']).flatten()
    kps = kps.astype(np.float32)

    kps[::2] *= 224.0 / w # adjust x coordinates
    kps[1::2] *= 224.0 / h # adjust y coordinates

    return img, kps



In [6]:
train_dataset = KeypointDataset('tennis_court_det_dataset/data/images', 'tennis_court_det_dataset/data/data_train.json')
val_dataset = KeypointDataset('tennis_court_det_dataset/data/images', 'tennis_court_det_dataset/data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [7]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # (x,y) coordinate for 14 set of points
model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Train Model

In [8]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 20

for epoch in range(epochs):
  for i, (imgs, kps) in enumerate(train_loader):
    imgs = imgs.to(device)
    kps = kps.to(device)

    optimizer.zero_grad() # flush out the gradients
    preds = model(imgs)
    loss = criterion(preds, kps)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
      print(f'epoch: {epoch+1} iter: {i} loss: {loss.item()}')

epoch: 1 iter: 0 loss: 15285.6220703125
epoch: 1 iter: 10 loss: 15551.7890625
epoch: 1 iter: 20 loss: 15223.9326171875
epoch: 1 iter: 30 loss: 13918.6259765625
epoch: 1 iter: 40 loss: 14085.0078125
epoch: 1 iter: 50 loss: 12149.818359375
epoch: 1 iter: 60 loss: 12670.8115234375
epoch: 1 iter: 70 loss: 12324.8251953125
epoch: 1 iter: 80 loss: 11983.5361328125
epoch: 1 iter: 90 loss: 11583.6162109375
epoch: 1 iter: 100 loss: 11303.5810546875
epoch: 1 iter: 110 loss: 10851.4990234375
epoch: 1 iter: 120 loss: 10155.978515625
epoch: 1 iter: 130 loss: 10007.4482421875
epoch: 1 iter: 140 loss: 9663.7705078125
epoch: 1 iter: 150 loss: 9756.8359375
epoch: 1 iter: 160 loss: 9054.4111328125
epoch: 1 iter: 170 loss: 8298.470703125
epoch: 1 iter: 180 loss: 8353.08984375
epoch: 1 iter: 190 loss: 8417.1669921875
epoch: 1 iter: 200 loss: 7484.42236328125
epoch: 1 iter: 210 loss: 7151.2255859375
epoch: 1 iter: 220 loss: 6894.482421875
epoch: 1 iter: 230 loss: 6909.294921875
epoch: 1 iter: 240 loss: 680

In [9]:
torch.save(model.state_dict(), 'tennis_court_keypoints.pth')